Question 1
We need to get the data from the file assets/companies_small_set.data into a DataFrame. The problem is that the data on each line of the file is in either a JSON or Tab-separated values (TSV) format.

The JSON lines are in the correct format, they just need to be converted to native Python dicts.

The TSV lines need to be converted in to dicts that match the JSON format.

Write a generator gen_fixed_data that takes an iterator as an arguement. It should parse the values in the iterator and yield each value in the correct format: A dict with the keys:

company
catch_phrase
phone
timezone
client_count

Note that your solution should be a generator function, it should not return a DataFrame.

In [ ]:
import json
import pandas as pd

def gen_fix_data(data_iterator):
    for line in data_iterator:
        line = line.strip()
        if line.startswith('{'):
            yield json.loads(line)
        else: 
            fields = line.split('\t')
            yield {
                "company": fields[0],
                "catch_phrase": fields[1],
                "phone": fields[2],
                "timezone": fields[3],
                "client_count": int(fields[4])
            }

        # YOUR CODE HERE
    #raise NotImplementedError()

**Explanation**
1. Import necessary libraries:

In [ ]:
import json
import pandas as pd

json: This module helps parse JSON formatted strings into Python dictionaries.

pandas as pd: This library is useful for data manipulation and analysis, but in this function, it’s mostly imported for later use when converting the parsed data to a DataFrame.

2. Define the generator function gen_fix_data:

In [ ]:
def gen_fix_data(data_iterator):

data_iterator: This parameter is an iterator that yields lines of data, each line being either JSON or TSV formatted.

3. Iterate over each line in the data iterator:

In [ ]:
for line in data_iterator:
    line = line.strip()

line.strip(): Removes any leading and trailing whitespace, including newlines, from the line.

4. Check if the line is in JSON format:

In [ ]:
if line.startswith('{'):
    yield json.loads(line)

line.startswith('{'): Determines if the line is JSON by checking if it starts with a {.

json.loads(line): Converts the JSON formatted string into a Python dictionary and yields it.

5. Handle TSV formatted lines:

In [ ]:
else:
    fields = line.split('\t')
    yield {
        "company": fields[0],
        "catch_phrase": fields[1],
        "phone": fields[2],
        "timezone": fields[3],
        "client_count": int(fields[4])
    }

line.split('\t'): Splits the TSV line into its respective fields using the tab character as the delimiter.

Constructing the dictionary: Creates a dictionary with the required keys (company, catch_phrase, phone, timezone, and client_count). The values are extracted from the split fields, and client_count is explicitly converted to an integer.

> So, the gen_fix_data generator function processes each line of data, determining its format, converting it to a consistent dictionary format, and yielding the resulting dictionaries. These can then be used to create a DataFrame or for other analyses.

Question 2
The data in assets/server_metrics.csv represents the time it take to handle requests in a start-up company's web application. Let's imagine we are asked to write some code that gives us a DataFrame that just contains the entries where processing_time is greater than 160 milliseconds.

We could solve that problem like this...

In [ ]:
df = pd.read_csv('assets/server_metrics.csv')

In [ ]:
outliers = df[df['processing_time'] > 160]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

_ = outliers['processing_time'].plot.hist(title="Times > 160")

But imagine that instead of dealing with millions of rows, we have to deal with billions or trillions and the set is too big to fit comfortably in memory, or that the data is coming to us not in a local file, but is being read over the network. Generators can be a nice way to help in that situation.

Here is a generator that yields a dict for each line in assets/server_metrics.csv.

Note that your solution should be a generator function, it should not return a DataFrame.

In [ ]:
def metrics_stream():
    '''
    Generate dictionaries from each line in assets/server_metrics.csv
    '''
    import csv

    with open('assets/server_metrics.csv', 'r') as stream:
        csv_stream = csv.DictReader(stream, ['job_id', 'processing_time', 'instance_id'])
        next(csv_stream) # throw away header row
        for entry in csv_stream:
            entry['processing_time'] = float(entry['processing_time'])
            yield dict(entry)

For this problem, write a generator that can be used to create a DataFrame like the outliers one above. Its first parameter should be the iterable we get from the metrics_stream() generator function. Its second (optional) parameter should be called lower_bound and be used to filter out entries whose "processing_time" is less than or equal to this parameter.

In [ ]:
def gen_outliers(metrics_iterable, lower_bound=160):
    for metric in metrics_iterable:
        if metric['processing_time'] > lower_bound:
            yield metric
    # YOUR CODE HERE
    # raise NotImplementedError()

**Explanation**

1. Function Definition:

In [ ]:
def gen_outliers(metrics_iterable, lower_bound=160):

This defines the generator function gen_outliers that takes two parameters: metrics_iterable (an iterable of metrics) and lower_bound (an optional parameter with a default value of 160).

2. Iteration and Filtering:

In [ ]:
for metric in metrics_iterable:
    if metric['processing_time'] > lower_bound:
        yield metric

This loop iterates through each item in metrics_iterable. For each metric, it checks if the processing_time is greater than lower_bound. If it is, it yields (returns) the metric.

3. Generating Outliers:

In [ ]:
metrics_gen = metrics_stream()
generated_outliers = pd.DataFrame(gen_outliers(metrics_gen))

Here, the metrics_stream generator creates an iterable of metrics, and gen_outliers processes this iterable to filter out only those metrics where processing_time is greater than 160. These filtered metrics are then converted into a pandas DataFrame named generated_outliers.

4. Plotting the Histogram:

In [ ]:
import matplotlib.pyplot as plt
_ = generated_outliers['processing_time'].plot.hist(title="Times > 160")
plt.show()

This code imports matplotlib.pyplot for plotting and then creates a histogram of the processing_time values in generated_outliers. The histogram is titled "Times > 160".

> In summary, this solution leverages a generator to efficiently filter out metrics with processing_time greater than 160 milliseconds and then creates a histogram to visualize these outliers. This approach is especially useful when dealing with large datasets, as it processes the data in a memory-efficient manner.

Question 3
Write a decorator called as_json that converts the wrapped function's return value to a JSON encoded string.

You can assume that this will only be used on functions whose return values can be converted to JSON.
This will be easiest if you use the standard library's json package.

In [ ]:
import json
from functools import wraps

def as_json(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return json.dumps(result)
    return wrapper

**Explanation**

1. Import Libraries:

In [ ]:
import json
from functools import wraps

json is imported for encoding data as JSON strings.

wraps is imported to ensure the wrapper function retains the metadata of the original function.

2. Define the Decorator:

In [ ]:
def as_json(func):

This is the definition of the as_json decorator. It takes a function func as its argument.

3. Define the Wrapper Function:

In [ ]:
@wraps(func)
def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)
    return json.dumps(result)

@wraps(func): This decorator ensures that the wrapper function retains the original function’s name, docstring, and other metadata.

def wrapper(*args, **kwargs): This defines the wrapper function which takes any number of positional and keyword arguments.

result = func(*args, **kwargs): Calls the original function func with the given arguments and stores the result.

return json.dumps(result): Converts the result to a JSON encoded string and returns it.

4. Return the Wrapper Function:

In [ ]:
return wrapper

> Now, you can use this decorator to convert the return value of any function to a JSON encoded string. 

In [ ]:
# Example
@as_json
def get_data():
    return {'name': 'Alice', 'age': 30}

print(get_data()) # Output: {"name": "Alice", "age": 30}

# This example demonstrates how the as_json decorator converts the dictionary 
# returned by get_data into a JSON string. 